# 概要

Juliaとawabi.jlによる形態素解の実験

## データの取得

In [25]:
using HTTP, JSON, CSV, TableOperations, DataFrames # ライブラリをインポート

"""
ニコニコ動画のデータを収集しファイルに保存する関数

参考: https://dwango.github.io/niconico/genre_ranking/ranking_log
"""
function collect_nico_data(ranking_type, date, file_type, endpoint="https://dcdn.cdn.nimg.jp/nicovideo/old-ranking")
    response = HTTP.get("$endpoint/$ranking_type/$date/$file_type")             # リクエストを送信
    response.status != 200 && (return error("Error: $(response.status)"))       # レスポンスが200以外の場合はエラーを返す
    data      = JSON.parse(String(response.body))                               # レスポンスをJSON形式に変換
    file_name = "$(ranking_type)_$(date)_$(file_type).csv"
    CSV.write(file_name, data; transform=((col, val)->something(val, missing))) # データをCSVファイルに保存（transformオプションを追加）
    return file_name
end

"""
ファイル名リストからタグを検索し、ファイル名を返す関数
"""
function search_tag(tag, file_name)
    file_name_list = CSV.read(file_name, DataFrame)
    # FIXME: ここでtagが見つからなかった場合の処理を追加する
    return file_name_list[coalesce.(file_name_list.tag, "") .== "アーマードコア6", :].file[1]
end

file_name_list_name = collect_nico_data("daily", "2023-09-23", "file_name_list.json")
ac6_file_name       = search_tag("アーマードコア6", file_name_list_name)
ac6_result_file     = collect_nico_data("daily", "2023-09-23", ac6_file_name)

"daily_2023-09-23_file_name_list.json.csv"

## 形態素解析の実験


In [16]:
using Awabi, PrettyPrint

tokenizer = Tokenizer(Dict("dicdir"=>"/home/linuxbrew/.linuxbrew/lib/mecab/dic/ipadic"))
text      = "すもももももももものうち"
tokens    = tokenize(tokenizer, text)
pprintln(tokens) # 結果を表示する


[
  ("すもも", "名詞,一般,*,*,*,*,すもも,スモモ,スモモ"),
  ("も", "助詞,係助詞,*,*,*,*,も,モ,モ"),
  ("もも", "名詞,一般,*,*,*,*,もも,モモ,モモ"),
  ("も", "助詞,係助詞,*,*,*,*,も,モ,モ"),
  ("もも", "名詞,一般,*,*,*,*,もも,モモ,モモ"),
  ("の", "助詞,連体化,*,*,*,*,の,ノ,ノ"),
  ("うち", "名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ"),
]
